### launching of SPs



In [1]:
from datetime import timedelta
import pandas as pd

import pyodbc
from sql.config import SQL_SERVER, DB_NAME
import sql.naming_convention as nc
from sql.sql_requests import SQL_Communicator

In [2]:
def format_time_delta(duration: timedelta):
    # Get the total seconds
    total_seconds = duration.total_seconds()
    # days = duration.days
    hours = duration.seconds // 3600  # Convert seconds to hours
    minutes = (duration.seconds // 60) % 60  # Convert remaining seconds to minutes
    seconds = duration.seconds % 60
    if hours > 0:
        return f'{hours:02d}:{minutes:02d}:{int(seconds):02d}'
    if minutes > 0:
        return f'{minutes:02d}:{int(seconds):02d}'
    return f'{total_seconds:4.2f}'

In [3]:
entity_names =  ['Symbol_IBESTickerEnhanced',
        'Symbol_TickerEnhanced',
        'Symbol_CusipEnhanced',
        'Symbol_FTCusip6Symbol',
        'Symbol_FTCusip9Symbol',
        'Symbol_FTISINSymbol',
        'Symbol_ISINEnhanced',
        'Symbol_SECCIKEnhanced',
        'Symbol_SECCIKSymbol',
        'Symbol_SedolEnhanced',
        'Symbol_FTSEDOLSymbol',]

durations = []
with SQL_Communicator() as sc:
    for en in entity_names:
        print(f'flow for target entity {en}..')
        en2 = nc.default_rename(en)
        sp_names = [nc.pull_sp_name(en2), nc.merge_sp_name(en2)]
        ent_durs, flow_td = [en2], timedelta()  
        for sp_name in sp_names:
            exec_stm = f"EXEC {sp_name}"
            print(f'- {exec_stm}...')
            td = sc.get_execution_metrics(exec_stm)
            ent_durs.append(format_time_delta(td))
            flow_td += td
        ent_durs.append(format_time_delta(flow_td))
        durations.append(ent_durs)
        # ent_durs.append(sum(ent_durs))
columns = ['Entity name', 'Pull SP', 'Merge SP',  'sum']
df = pd.DataFrame(durations, columns = columns)


connecting...
Connection successful!
flow for target entity Symbol_IBESTickerEnhanced..
- EXEC dbo.PullData_Symbol_IBESTickerEnhanced2_prc...
- EXEC dbo.MergeData_Symbol_IBESTickerEnhanced2_prc...
['Symbol_IBESTickerEnhanced2', '0.41', '1.07', '1.48']
flow for target entity Symbol_TickerEnhanced..
- EXEC dbo.PullData_Symbol_TickerEnhanced2_prc...
- EXEC dbo.MergeData_Symbol_TickerEnhanced2_prc...
['Symbol_TickerEnhanced2', '29.88', '34.77', '01:04']
flow for target entity Symbol_CusipEnhanced..
- EXEC dbo.PullData_Symbol_CusipEnhanced2_prc...
- EXEC dbo.MergeData_Symbol_CusipEnhanced2_prc...
['Symbol_CusipEnhanced2', '25.70', '29.43', '55.13']
flow for target entity Symbol_FTCusip6Symbol..
- EXEC dbo.PullData_Symbol_FTCusip6Symbol2_prc...
- EXEC dbo.MergeData_Symbol_FTCusip6Symbol2_prc...
['Symbol_FTCusip6Symbol2', '3.19', '2.97', '6.16']
flow for target entity Symbol_FTCusip9Symbol..
- EXEC dbo.PullData_Symbol_FTCusip9Symbol2_prc...
- EXEC dbo.MergeData_Symbol_FTCusip9Symbol2_prc...
[

In [4]:
df

,Entity name,Pull SP,Merge SP,sum
0,Symbol_IBESTickerEnhanced2,0.41,1.07,1.48
1,Symbol_TickerEnhanced2,29.88,34.77,01:04
2,Symbol_CusipEnhanced2,25.70,29.43,55.13
3,Symbol_FTCusip6Symbol2,3.19,2.97,6.16
4,Symbol_FTCusip9Symbol2,19.16,11.67,30.82
5,Symbol_FTISINSymbol2,50.19,58.97,01:49
6,Symbol_ISINEnhanced2,59.90,01:16,02:15
7,Symbol_SECCIKEnhanced2,3.61,5.47,9.08
8,Symbol_SECCIKSymbol2,4.00,2.59,6.59
9,Symbol_SedolEnhanced2,02:24,02:39,05:03
